Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved BSD-3 license. (c) Lorena A. Barba, Gilbert F. Forsyth 2017. Thanks to NSF for support via CAREER award #1149784.
<p><a href="https://twitter.com/LorenaABarba">@LorenaABarba</a></p>
<p>This lesson complements the online CFDPython class, by Prof. Lorena A. Barba called <b>12 Steps to Navier-Stokes</b>.  It was written by Prof. Barba with BU Graduate Student Gilbert Forsyth.</p>
<p>  It has been modified by BBFlyer1 as part of the CFDPython Class.</p>

<h1>Array Operations with Numpy</h1>
<p>Computation using arrays can be performed using for loops.  However for larger arrays, using the Numpy Array operations can significantly increase execution speed many times over. Here is a simple example using the following equation.  </p>
<img class="math math-display" alt="$$u^{n+1}_i = u^n_i-u^n_{i-1}$$" src="https://render.githubusercontent.com/render/math?math=u%5E%7Bn%2B1%7D_i%20%3D%20u%5En_i-u%5En_%7Bi-1%7D&amp;mode=display">
<p>Now given a vector $u^n=[0,1,2,3,4,5]\\$ we can calculate the various values of U^(n+1) by iterating over the values of Un with a loop.

In [2]:
import numpy

u = numpy.array((0,1,2,3,4,5))

for i in range(1, len(u)):
    print(u[i] - u[i-1])

1
1
1
1
1


This is the expected result and the execution was nearly instantaneous.  If the same operation is done using the array operation rather than calculating the differences 5 separate times as above, we can get the result with one operation.

In [3]:
u[1:] - u[0:-1]

array([1, 1, 1, 1, 1])

What this command says is to subtract the 0th, 1st, 2nd, 3rd, 4th, and 5th element of u from the 1st, 2nd, 3rd, 4th, 5th and 6th element of u.  
<p><b>Speed Increases</b></p>
On a 6 element array, on my Windows 10 computer, this array computation was visibly faster than the for loop calculation.  When the computations are for 100s or 1000s of elements, this speed enhancement will make a substantial difference in whether the calculation is even feasible or not.  Now lets take a look at the speed difference of a 2D linear convection case.   First we will assign our variables.

In [5]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2/(nx-1)
dy = 2/(ny-1)
sigma = 0.2
dt = sigma * dx

x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)

u = numpy.ones((ny, nx)) ##Create a 1xn, vector of 1's
un = numpy.ones((ny, nx)) ##Create a 1yn, vector of 1's

### Assign initial conditions
u[int(0.5 / dy): int(1 / dy +1), int(0.5 / dx): int(1 / dx +1) ] = 2

With our initial conditions now set, let's first try running our original nested loop code and timing it with the iPython "magic" function <code>%%timeit</code>, which will help evaluate the speed of our code.
<p><strong>Note:</strong> The <code>%%timeit</code> magic function will run the code several times and compute an average time as a result.  If you have figures or other outputs inside the timed code, these will plot or print repeatly which can be a bit messy.</p>

The execution times given below vary from machine to machine and are unlikely to match those of your machine.  However, you should see some reduction in the time required when we switch to computation using arrays.

In [7]:
%%timeit
#Initialize our array u
u = numpy.ones((ny, nx))
u[int(0.5 / dy): int(1 / dy +1), int(0.5 / dx): int(1 / dx +1) ] = 2

for n in range (nt + 1): ##loop across the number of time steps.
    un = u.copy()
    row, col = u.shape
    for j in range (1,row):
        for i in range(1,col):
            u[j,i] = (u[j,i] - (c * dt / dx *
                                (un[j,i] - un[j,i-1]))-
                                (c * dt / dy *
                                 (un[j,i] - un[j-1,i])))
        u[0, :]  = 1
        u[-1, :] = 1
        u[:, 0]  = 1
        u[:, -1]  = 1

2 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


When the "raw" Python code above, the mean execution on my Windows 10 laptop was ~2 seconds mean for 7 loops, a little slower than Dr. Barba's example which ran a best of 1.94 seconds for 3 loops.  Now let us compare the speed of this computation using arrays instead of nested loops.

In [11]:
%%timeit
#Initialize our array u
u = numpy.ones((ny, nx))
u[int(0.5 / dy): int(1 / dy +1), int(0.5 / dx): int(1 / dx + 1) ] = 2

for n in range(nt + 1): ##loop across the number of time steps.
    un = u.copy()
    u[1:, 1:] = un[1:, 1:] -((c * dt / dx * (un[1:, 1:] - un[1:, 0:-1]))-
                             (c * dt / dy * (un[1:, 1:] - un[0:-1, 1:])))
    u[0, :]  = 1
    u[-1, :] = 1
    u[:, 0]  = 1
    u[:, -1]  = 1

14.4 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


As you can see, the 14.4 ms speed increase for the array function was a significant over the 2 seconds "for" loop.  This was very good, even though it was significantly slower than the 5.09 ms for Dr. Barba's results.  2 seconds is not a huge amount of time, but these speeds will increase exponentially as the size and complexity of the problem being evaluated increase.

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()